In [1]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import numpy as np

In [2]:
df = pd.read_csv("raw_ksi_data.csv")

In [3]:
df.shape

(18957, 54)

In [4]:
df.sample(2)

,OBJECTID,INDEX,ACCNUM,DATE,TIME,STREET1,STREET2,OFFSET,ROAD_CLASS,DISTRICT,...,REDLIGHT,ALCOHOL,DISABILITY,HOOD_158,NEIGHBOURHOOD_158,HOOD_140,NEIGHBOURHOOD_140,DIVISION,x,y
14925,14926,80950560,NaN,9/11/2018 8:00:00 AM,1708,LAKE SHORE BLVD W,SIXTH ST,NaN,Major Arterial,Etobicoke York,...,NaN,NaN,NaN,18,New Toronto,18,New Toronto (18),D22,620736.2663,4.828634e+06
1831,1832,4163615,966448.0,4/5/2007 8:00:00 AM,1305,UNIVERSITY AVE,WELLINGTON ST W,NaN,Major Arterial,Toronto and East York,...,NaN,NaN,NaN,170,Yonge-Bay Corridor,76,Bay Street Corridor (76),D52,630322.3289,4.833862e+06


In [5]:
df.drop(['ACCNUM','OBJECTID', 'HOOD_140', 'NEIGHBOURHOOD_140', 'OFFSET', 'ROAD_CLASS', 'ACCLOC', 'INVAGE', 'INITDIR'], axis=1, inplace=True)

In [6]:
df.drop(['MANOEUVER', 'DRIVCOND', 'PEDTYPE', 'PEDACT', 'CYCLISTYPE', 'CYCACT', 'CYCCOND', 'PEDESTRIAN', "TRSN_CITY_VEH"], axis=1, inplace=True)

In [7]:
df.drop(['EMERG_VEH', 'PASSENGER', 'REDLIGHT', 'DISABILITY', 'DIVISION', 'SPEEDING', 'PEDCOND', 'AG_DRIV','x', 'y' ], axis=1, inplace=True)

In [8]:
df.drop(['CYCLIST', 'INVTYPE', 'INDEX', 'TRAFFCTL', 'AUTOMOBILE', 'MOTORCYCLE', 'TRUCK', 'ALCOHOL'], axis=1, inplace=True)

In [9]:
df.shape

(18957, 18)

In [10]:
df.sample(2)

,DATE,TIME,STREET1,STREET2,DISTRICT,LATITUDE,LONGITUDE,VISIBILITY,LIGHT,RDSFCOND,ACCLASS,IMPACTYPE,INJURY,FATAL_NO,VEHTYPE,DRIVACT,HOOD_158,NEIGHBOURHOOD_158
17010,4/8/2021 8:00:00 AM,1950,MORNINGSIDE AVE,MILNER AVE,Scarborough,43.800208,-79.199046,Clear,"Dusk, artificial",Dry,Non-Fatal Injury,Turning Movement,Major,NaN,Motorcycle,Driving Properly,144,Morningside Heights
13275,1/19/2017 10:00:00 AM,1755,COLLINSGROVE RD,LAWRENCE AV E,Scarborough,43.769919,-79.183154,Clear,"Dark, artificial",Dry,Non-Fatal Injury,Pedestrian Collisions,Major,NaN,NaN,NaN,136,West Hill


In [11]:
df['TIME'] = df['TIME'].astype(str).str.zfill(4)
df['DATE'] = pd.to_datetime(df['DATE'], format='%m/%d/%Y %I:%M:%S %p')

# Extract DOW and HOUR and YEAR
df['DOW'] = df['DATE'].dt.day_name() 
df['HOUR'] = df['TIME'].str[:2].astype(int)
df['YEAR'] = df['DATE'].dt.year
df['MONTH'] = df['DATE'].dt.month

In [12]:
df.sample(3)

,DATE,TIME,STREET1,STREET2,DISTRICT,LATITUDE,LONGITUDE,VISIBILITY,LIGHT,RDSFCOND,...,INJURY,FATAL_NO,VEHTYPE,DRIVACT,HOOD_158,NEIGHBOURHOOD_158,DOW,HOUR,YEAR,MONTH
18545,2023-07-11 08:00:00,2029,EGLINTON AVE W,KANE AVE,Toronto and East York,43.691306,-79.470020,Clear,Daylight,Dry,...,NaN,NaN,"Automobile, Station Wagon",Driving Properly,112,Beechborough-Greenbrook,Tuesday,20,2023,7
241,2006-03-19 10:00:00,1743,STEELES AVE E,WARDEN AVE,Scarborough,43.819645,-79.324390,Clear,Daylight,Dry,...,Major,NaN,"Automobile, Station Wagon",Driving Properly,NSA,NSA,Sunday,17,2006,3
12913,2016-09-05 08:00:00,0210,ST CLAIR AVE E,DANFORTH RD,Scarborough,43.716773,-79.259883,Clear,"Dark, artificial",Dry,...,Major,NaN,NaN,NaN,120,Clairlea-Birchmount,Monday,2,2016,9


In [13]:
df["INJURY"].unique()

array(['Major', 'Minor', nan, 'Fatal', 'Minimal'], dtype=object)

In [14]:
df["INJURY"].value_counts()

INJURY
Major      6445
Minor      1479
Minimal    1160
Fatal       976
Name: count, dtype: int64

In [15]:
df['INJURY'] = df['INJURY'].fillna('Minimal')

In [16]:
df["ACCLASS"].unique()

array(['Non-Fatal Injury', 'Fatal', 'Property Damage O', nan],
      dtype=object)

In [17]:
df["ACCLASS"].value_counts()

ACCLASS
Non-Fatal Injury     16268
Fatal                 2670
Property Damage O       18
Name: count, dtype: int64

In [18]:
df['ACCLASS'] = df['ACCLASS'].replace('Property Damage O', 'Non-Fatal Injury')
df['ACCLASS'] = df['ACCLASS'].fillna('Non-Fatal Injury')

In [19]:
df['SEVERE_COLLISION'] = (
    (df['INJURY'].isin(['Major', 'Fatal'])) | 
    (df['ACCLASS'] == 'Fatal')
).astype(int)

In [20]:
df['FATAL_NO'] = df['FATAL_NO'].fillna(0)

In [21]:
df["VISIBILITY"].value_counts()

VISIBILITY
Clear                     16373
Rain                       1976
Snow                        356
Other                        98
Fog, Mist, Smoke, Dust       52
Freezing Rain                47
Drifting Snow                21
Strong wind                  10
Name: count, dtype: int64

In [22]:
df['VISIBILITY'] = df['VISIBILITY'].replace({
    'Other': 'Clear',
    'Freezing Rain': 'Rain',
    'Strong wind': 'Rain',
    'Fog, Mist, Smoke, Dust': 'Rain',
    'Drifting Snow': 'Snow'
})

In [23]:
df["VISIBILITY"].value_counts()

VISIBILITY
Clear    16471
Rain      2085
Snow       377
Name: count, dtype: int64

In [24]:
df["LIGHT"].value_counts()

LIGHT
Daylight                10779
Dark                     3746
Dark, artificial         3552
Dusk                      253
Dusk, artificial          253
Daylight, artificial      151
Dawn                      112
Dawn, artificial          101
Other                       6
Name: count, dtype: int64

In [25]:
df['LIGHT'] = df['LIGHT'].replace({
    'Daylight, artificial': 'Daylight',
    'Dark, artificial': 'Dark',
    'Dusk, artificial': 'Twilight',
    'Dawn, artificial': 'Twilight',
    'Dawn': 'Twilight',
    'Dusk': 'Twilight',
    'Other': 'Daylight'
})

In [26]:
df["LIGHT"].value_counts()

LIGHT
Daylight    10936
Dark         7298
Twilight      719
Name: count, dtype: int64

In [27]:
df.rename(columns={'RDSFCOND': 'ROAD_CONDITION'}, inplace=True)

In [28]:
df["ROAD_CONDITION"].value_counts()

ROAD_CONDITION
Dry                     15231
Wet                      3140
Loose Snow                174
Other                     147
Slush                     102
Ice                        78
Packed Snow                44
Loose Sand or Gravel       11
Spilled liquid              1
Name: count, dtype: int64

In [29]:
df['ROAD_CONDITION'] = df['ROAD_CONDITION'].replace({
    'Spilled liquid': 'Wet',
    'Loose Snow': 'SnowIce',
    'Packed Snow': 'SnowIce',
    'Slush': 'SnowIce',
    'Ice': 'SnowIce',
    'Loose Sand or Gravel': 'Dry',
    'Other': 'Dry'
})

In [30]:
df["ROAD_CONDITION"].value_counts()

ROAD_CONDITION
Dry        15389
Wet         3141
SnowIce      398
Name: count, dtype: int64

In [31]:
df['DATE'] = pd.to_datetime(df['DATE']).dt.date

In [32]:
df.drop(['TIME'], axis=1, inplace=True)#since hour col, exact time not needed

In [33]:
df.sample(5)

,DATE,STREET1,STREET2,DISTRICT,LATITUDE,LONGITUDE,VISIBILITY,LIGHT,ROAD_CONDITION,ACCLASS,...,FATAL_NO,VEHTYPE,DRIVACT,HOOD_158,NEIGHBOURHOOD_158,DOW,HOUR,YEAR,MONTH,SEVERE_COLLISION
12800,2016-08-01,MORNINGSIDE AVE,HALFWAY AVE,Scarborough,43.817847,-79.213687,Clear,Dark,Dry,Non-Fatal Injury,...,0.0,NaN,NaN,146,Malvern East,Monday,1,2016,8,0
4502,2009-04-20,RONCESVALLES AVE,HIGH PARK BLVD,Toronto and East York,43.646327,-79.449046,Rain,Daylight,Wet,Fatal,...,0.0,Municipal Transit Bus (TTC),Failed to Yield Right of Way,86,Roncesvalles,Monday,13,2009,4,1
13895,2017-08-28,5450 DUNDAS ST W,NaN,Etobicoke York,43.632731,-79.543445,Clear,Daylight,Dry,Non-Fatal Injury,...,0.0,NaN,NaN,159,Etobicoke City Centre,Monday,8,2017,8,1
8863,2012-10-21,MCNICOLL AVE,BIRCHMOUNT RD,Scarborough,43.807944,-79.314189,Clear,Dark,Dry,Fatal,...,0.0,Other,NaN,116,Steeles,Sunday,3,2012,10,1
8804,2012-10-11,OSSINGTON AVE,DUNDAS ST W,Toronto and East York,43.649321,-79.420772,Clear,Daylight,Dry,Non-Fatal Injury,...,0.0,Taxi,Improper Turn,81,Trinity-Bellwoods,Thursday,7,2012,10,0


In [34]:
def categorize_hour(hour):
    if 0 <= hour < 8:
        return 'LateNight'
    elif 8 <= hour < 14:
        return 'MorningAfternoon'
    elif 14 <= hour < 19:
        return 'Evening'
    else:
        return 'Night'

In [35]:
def categorize_season(month):
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    else:
        return 'Fall'

In [36]:
def categorize_vehicle(v):
    if v in ["Automobile, Station Wagon", "Passenger Van", "Taxi", "Other"]:
        return "Car"
    elif v in [
        "Pick Up Truck", "Truck – Open", "Truck – Closed (Blazer, etc)",
        "Truck – Dump", "Truck–Tractor", "Delivery Van",
        "Tow Truck", "Truck (other)", "Truck – Car Carrier",
        "Construction Equipment"
    ]:
        return "Truck"
    elif v in [
        "Municipal Transit Bus (TTC)", "Bus (Other) (Go Bus, Gray Coa)",
        "Intercity Bus", "School Bus"
    ]:
        return "Bus"
    elif v in ["Motorcycle", "Bicycle", "Moped", "Rickshaw"]:
        return "Two Wheeler"
    elif v in ["Police Vehicle", "Fire Vehicle", "Ambulance", "Other Emergency Vehicle"]:
        return "Emergency"
    else:
        return "Car"

In [37]:
def categorize_drivact(a):
    if a == "DrivingProperly":
        return "Driver Proper"
    elif a in ["Failed to Yield Right of Way", "Disobeyed Traffic Control", "Wrong Way on One Way Road"]:
        return "Right-of-Way/Traffic Violation"
    elif a in ["Exceeding Speed Limit", "Speed too Fast For Condition", "Speed too Slow"]:
        return "Speed-Related"
    elif a in ["Lost control", "Improper Turn", "Following too Close", "Improper Lane Change", "Improper Passing"]:
        return "Control/Maneuver Error"
    else:
        return "Other/Unknown"


In [38]:
def categorize_impact(impact):
    if impact in ["Pedestrian Collisions", "Cyclist Collisions"]:
        return "Pedestrian/Cyclist"
    elif impact == "Rear End":
        return "Rear End"
    elif impact in ["Sideswipe", "Approaching"]:
        return "Sideswipe/Approaching"
    elif impact in ["Angle", "Turning Movement"]:
        return "T-Bone"
    elif impact in ["SMV Other", "SMV Unattended Vehicle"]:
        return "Slow/Parked"
    else:
        return "Other/Unknown"

In [39]:
df['TIME_OF_DAY'] = df['HOUR'].apply(categorize_hour)
df['SEASON'] = df['MONTH'].apply(categorize_season)
df["VEHICLE_TYPE"] = df["VEHTYPE"].apply(categorize_vehicle)
df["DRIVER_ACTION"] = df["DRIVACT"].apply(categorize_drivact)
df["IMPACT_TYPE"] = df["IMPACTYPE"].apply(categorize_impact)

In [40]:
df = df[df["IMPACT_TYPE"] != "Other/Unknown"]

In [41]:
df['ROAD_CONDITION'] = df['ROAD_CONDITION'].replace(['', ' ', 'nan', 'NaN'], np.nan)
df.loc[df['ROAD_CONDITION'].isna() & (df['SEVERE_COLLISION'] == 1), 'ROAD_CONDITION'] = 'Wet'
df.loc[df['ROAD_CONDITION'].isna() & (df['SEVERE_COLLISION'] == 0), 'ROAD_CONDITION'] = 'Dry'

In [42]:
df['LIGHT'] = df['LIGHT'].replace(['', ' ', 'nan', 'NaN'], np.nan)
df.loc[df['LIGHT'].isna() & (df['SEVERE_COLLISION'] == 1), 'LIGHT'] = 'Dark'
df.loc[df['LIGHT'].isna() & (df['SEVERE_COLLISION'] == 0), 'LIGHT'] = 'Daylight'

In [43]:
df['VISIBILITY'] = df['VISIBILITY'].replace(['', ' ', 'nan', 'NaN'], np.nan)
df.loc[df['VISIBILITY'].isna() & (df['SEVERE_COLLISION'] == 1), 'VISIBILITY'] = 'Rain'
df.loc[df['VISIBILITY'].isna() & (df['SEVERE_COLLISION'] == 0), 'VISIBILITY'] = 'Clear'

In [44]:
df.shape

(18726, 27)

In [45]:
df.to_csv('cleaned_ksi_collisions.csv', index=False)